In [141]:
import praw
import pandas as pd
import re
import string
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/keras/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [142]:
# get reddit secret & script i've saved as .txt in the working directory

with open('reddit_oa_script.txt') as f:
    
    reddit_script = f.read().strip()
    
with open('reddit_oa_secret.txt') as f:
    
    reddit_secret = f.read().strip()
    
with open('reddit_id.txt') as f:
    
    pw = f.read().strip()

In [143]:
# user_agent is the name of the app

reddit  = praw.Reddit(client_id=reddit_script,
                          client_secret=reddit_secret,
                          user_agent='get_corpora',
                          username='neurodivergent_ai',
                          password=pw)


In [144]:
# get the subreddit

subreddit_1 = reddit.subreddit('The_Donald')

# get the top (upvoted) 1000 threads

subreddit_1_top = subreddit_1.top(limit=1000)

In [145]:
# get the top 5 submissions to check

for post in subreddit_1.top(limit=5):
    
    print(post.title, '\n')

DONALD J. TRUMP DECLARED THE WINNER! 

I’m Donald J. Trump and I'm Your Next President of the United States. 

The New Algorithm Was Designed To Keep President Trump From The Front Page... But Sadly, That Won't Happen. 

CNN will soon be #1...when searching for the term "Fake News". Congratulations to CNN on this major achievement. 

ALL CELEBRITIES THAT VOWED TO LEAVE THE U.S.A. IF TRUMP WINS, WE NOW DEMAND YOU TO FOLLOW THROUGH!!!! OUT! OUT! OUT! 



In [146]:
# build a dataset

# choose information to gather

columns_dict = {'title':[],
               'id':[],
               'url':[],
               'date':[],
               'score':[],
               'num_comments':[],
               'body':[]}

# build a dictionary

for post in subreddit_1_top:
    
    columns_dict['title'].append(post.title)
    
    columns_dict['id'].append(post.id)
    
    columns_dict['url'].append(post.url)
    
    columns_dict['date'].append(post.created)
    
    columns_dict['score'].append(post.score)
    
    columns_dict['num_comments'].append(post.num_comments)
    
    columns_dict['body'].append(post.selftext)

# convert dictionary to a dataframe

subreddit_1_df = pd.DataFrame(columns_dict)

In [147]:
subreddit_1_df.head()

,title,id,url,date,score,num_comments,body
0,DONALD J. TRUMP DECLARED THE WINNER!,5bzjv5,https://www.reddit.com/r/The_Donald/comments/5...,1.478706e+09,66714,12097,#Meme Magic is REAL.\n\n#Donald J. Trump is th...
1,I’m Donald J. Trump and I'm Your Next Presiden...,4uxdbn,https://www.reddit.com/r/The_Donald/comments/4...,1.469687e+09,42118,20806,Hello The_Donald readers and the entire Reddit...
2,The New Algorithm Was Designed To Keep Preside...,5gvy1j,https://i.sli.mg/gu0oHA.png,1.481092e+09,40966,835,
3,CNN will soon be #1...when searching for the t...,5jt9xs,https://i.redd.it/7jeyhzgku65y.png,1.482472e+09,37276,724,
4,ALL CELEBRITIES THAT VOWED TO LEAVE THE U.S.A....,5bz5ds,https://www.reddit.com/r/The_Donald/comments/5...,1.478702e+09,36614,1685,Now that we have proven our commitment to the ...


In [148]:
# create a copy to work on

reddit_df = subreddit_1_df.copy()

In [149]:
# grab texts to build into corpus

titles_df = reddit_df['title']

posts_df = reddit_df['body']

In [150]:
titles_df.head(10)

0                 DONALD J. TRUMP DECLARED THE WINNER!
1    I’m Donald J. Trump and I'm Your Next Presiden...
2    The New Algorithm Was Designed To Keep Preside...
3    CNN will soon be #1...when searching for the t...
4    ALL CELEBRITIES THAT VOWED TO LEAVE THE U.S.A....
5                             IMMINENT VICTORY THREAD.
6    Hey admins, we found a picture of your wife's ...
7             Oh yeah, nothing fucky going on here :^)
8                     Can't Stump your President Trump
9    It's official: Trump will become the first U.S...
Name: title, dtype: object

In [151]:
posts_df.head(10)

0    #Meme Magic is REAL.\n\n#Donald J. Trump is th...
1    Hello The_Donald readers and the entire Reddit...
2                                                     
3                                                     
4    Now that we have proven our commitment to the ...
5    #THIS IS NOT A DRILL. WE ARE ACTUALLY GOING TO...
6                                                     
7                                                     
8                                                     
9                                                     
Name: body, dtype: object

In [152]:
# experiment
# need to toeknize, etc

text = posts_df[4].split()

print(text)

['Now', 'that', 'we', 'have', 'proven', 'our', 'commitment', 'to', 'the', 'cause,', 'now', 'that', 'we', 'have', 'voiced', 'and', 'acted', 'on', 'our', 'patriotism,', 'now', 'that', 'against', 'all', 'odds', 'and', 'constant', 'criticism', 'with', 'the', 'world', 'against', 'us,', 'we', 'have', 'won,', 'and', 'we', 'call', 'for', 'this:', '#All', 'of', 'you', 'elitist', 'hollywood', 'celebrities', 'who', 'vowed', 'to', 'betray', 'your', 'country', 'and', 'leave,', 'to', 'abandon', 'and', 'shun', 'those', 'who', 'you', 'spent', 'your', 'careers', 'milking', 'for', 'your', 'posh', 'lifestyles,', 'we', 'ask,', 'nay,', 'demand', 'that', 'you', 'follow', 'through', 'on', 'your', 'claims', 'to', 'leave', 'the', 'country.', 'We', 'as', 'Americans', 'have', 'spoken,', 'and', 'you', 'brushed', 'us', 'and', 'our', 'movement', 'off,', 'as', 'if', 'you', 'were', 'somehow', 'above', 'us,', 'forgetting', 'that', 'it', 'was', 'us', 'who', 'made', 'you.', '**FUCK', 'YOU!**', 'You', 'think', 'you', 'ar

the text needs to be cleaned. lots of stopwords, punctuation, & non-alpha characters. case needs to be lowered etc.

In [153]:
def clean_text(doc):
    '''
    tokenizes input text, converts to lowercase,
    removes punctuation/non-alphabetic characters
    and stopwords. returns clean document.
    
    '''
    
    # tokenize
    
    split_tokens = doc.split()
    
    # grab punctuation list
    
    punctuation = string.punctuation
    
    # set up regex 
    
    punct_filter = re.compile('[%s]' % re.escape(punctuation))
    
    # filter out punctuation
    
    no_punct_tokens = [punct_filter.sub('', char) for char in split_tokens]
    
    # remove non-alphabetic tokens
    
    alpha_tokens = [t for t in no_punct_tokens if t.isalpha()]
    
    # lowercase
    
    lower_tokens = [t.lower() for t in alpha_tokens]
    
    # get stop words
    
    stop_words = set(stopwords.words('english'))
    
    # filter out stopwords
    
    go_tokens = [t for t in lower_tokens if not t in stop_words]
    
    # remove very short tokens
    
    clean_tokens = [word for word in go_tokens if len(word) > 1]
    
    return clean_tokens
    

In [154]:
# test

raw_text = posts_df[4]

clean_text(raw_text)

['proven',
 'commitment',
 'cause',
 'voiced',
 'acted',
 'patriotism',
 'odds',
 'constant',
 'criticism',
 'world',
 'us',
 'call',
 'elitist',
 'hollywood',
 'celebrities',
 'vowed',
 'betray',
 'country',
 'leave',
 'abandon',
 'shun',
 'spent',
 'careers',
 'milking',
 'posh',
 'lifestyles',
 'ask',
 'nay',
 'demand',
 'follow',
 'claims',
 'leave',
 'country',
 'americans',
 'spoken',
 'brushed',
 'us',
 'movement',
 'somehow',
 'us',
 'forgetting',
 'us',
 'made',
 'fuck',
 'think',
 'good',
 'us',
 'well',
 'good',
 'follow',
 'vapid',
 'empty',
 'threat',
 'mistakenly',
 'pridefully',
 'consider',
 'get',
 'fuck',
 'fame',
 'celebrity',
 'status',
 'despicable',
 'shall',
 'say',
 'deplorable',
 'better',
 'us',
 'important',
 'us',
 'would',
 'nothing',
 'without',
 'us',
 'dare',
 'pretend',
 'us',
 'list',
 'demand',
 'leave',
 'asap',
 'amy',
 'unfunny',
 'schumer',
 'lena',
 'dunham',
 'barbra',
 'streisand',
 'bryan',
 'cranston',
 'miley',
 'cyrus',
 'amy',
 'schumer',


In [155]:
def clean_all_texts(texts):
    '''
    iterate through a list of texts
    apply clean_text() function to each
    append each clean text to a list
    return the list of cleaned texts
    
    '''
    
    cleaned_texts = []
    
    for text in texts:
        
        text = clean_text(text)
        
        cleaned_texts.append(text)
        
    return cleaned_texts

In [156]:
clean_titles = clean_all_texts(titles)

clean_posts = clean_all_texts(posts)

In [166]:
# test

print(clean_titles[4])

print('\n')

print(clean_posts[4])

['celebrities', 'vowed', 'leave', 'usa', 'trump', 'wins', 'demand', 'follow']


['proven', 'commitment', 'cause', 'voiced', 'acted', 'patriotism', 'odds', 'constant', 'criticism', 'world', 'us', 'call', 'elitist', 'hollywood', 'celebrities', 'vowed', 'betray', 'country', 'leave', 'abandon', 'shun', 'spent', 'careers', 'milking', 'posh', 'lifestyles', 'ask', 'nay', 'demand', 'follow', 'claims', 'leave', 'country', 'americans', 'spoken', 'brushed', 'us', 'movement', 'somehow', 'us', 'forgetting', 'us', 'made', 'fuck', 'think', 'good', 'us', 'well', 'good', 'follow', 'vapid', 'empty', 'threat', 'mistakenly', 'pridefully', 'consider', 'get', 'fuck', 'fame', 'celebrity', 'status', 'despicable', 'shall', 'say', 'deplorable', 'better', 'us', 'important', 'us', 'would', 'nothing', 'without', 'us', 'dare', 'pretend', 'us', 'list', 'demand', 'leave', 'asap', 'amy', 'unfunny', 'schumer', 'lena', 'dunham', 'barbra', 'streisand', 'bryan', 'cranston', 'miley', 'cyrus', 'amy', 'schumer', 'jon', 'stew

In [164]:
# get the first 25 titles

counter = 0

for i in clean_titles:
    
    if counter < 25:
        
        print(i, '\n')
        
        counter += 1

['donald', 'trump', 'declared', 'winner'] 

['donald', 'trump', 'im', 'next', 'president', 'united', 'states'] 

['new', 'algorithm', 'designed', 'keep', 'president', 'trump', 'front', 'page', 'sadly', 'wont', 'happen'] 

['cnn', 'soon', 'searching', 'term', 'fake', 'news', 'congratulations', 'cnn', 'major', 'achievement'] 

['celebrities', 'vowed', 'leave', 'usa', 'trump', 'wins', 'demand', 'follow'] 

['imminent', 'victory', 'thread'] 

['hey', 'admins', 'found', 'picture', 'wifes', 'boyfriends', 'president'] 

['oh', 'yeah', 'nothing', 'fucky', 'going'] 

['cant', 'stump', 'president', 'trump'] 

['official', 'trump', 'become', 'first', 'us', 'president', 'support', 'gay', 'marriage', 'starting', 'inauguration', 'day', 'shame', 'reached', 'rall'] 

['fuck', 'reddit', 'admins', 'like', 'redit', 'try', 'much', 'like', 'cant', 'stop', 'portrait', 'getting', 'upvotes', 'deserves'] 

['reddit', 'cuck', 'admins', 'take', 'official', 'portrait', 'president', 'upvoted', 'post', 'reddits', '

In [165]:
# get the first 25 posts

counter = 0

for i in clean_posts:
    
    if counter < 25:
        
        print(i, '\n')
        
        counter += 1

['meme', 'magic', 'real', 'donald', 'trump', 'next', 'president', 'united', 'states', 'america', 'let', 'tendies', 'hit', 'floor', 'donald', 'trump', 'proved', 'entire', 'world', 'wrong', 'insurmountable', 'odds', 'corruption', 'media', 'sabotage', 'party', 'crush', 'establishment', 'machine', 'donald', 'trump', 'feel', 'centipedes', 'god', 'emperor', 'said', 'would', 'get', 'tired', 'winning', 'tired', 'winning', 'yet', 'feel', 'vindicated', 'centipedes', 'nothing', 'future', 'history', 'part', 'history', 'thedonald', 'part', 'history', 'knew', 'hearts', 'one', 'believed', 'us', 'always', 'knew', 'truth', 'brakes', 'train', 'trump', 'wins'] 

['hello', 'thedonald', 'readers', 'entire', 'reddit', 'community', 'going', 'huge', 'im', 'looking', 'forward', 'answering', 'questions', 'im', 'flight', 'visit', 'great', 'people', 'toledo', 'oh', 'internet', 'connection', 'might', 'spotty', 'promise', 'ill', 'answer', 'questions', 'want', 'big', 'things', 'america', 'president', 'make', 'americ

there are no missing lines in `clean_titles`, since every post must at least have a title. some posts, however, consist only of the title (with no body). 

because of this, there are a number of empty rows in `clean_posts`.

In [167]:
clean_posts_2 = [i for i in clean_posts if len(i) > 0]

In [168]:
counter = 0

for i in clean_posts_2:
    
    if counter < 25:
        
        print(i, '\n')
        
        counter += 1

['meme', 'magic', 'real', 'donald', 'trump', 'next', 'president', 'united', 'states', 'america', 'let', 'tendies', 'hit', 'floor', 'donald', 'trump', 'proved', 'entire', 'world', 'wrong', 'insurmountable', 'odds', 'corruption', 'media', 'sabotage', 'party', 'crush', 'establishment', 'machine', 'donald', 'trump', 'feel', 'centipedes', 'god', 'emperor', 'said', 'would', 'get', 'tired', 'winning', 'tired', 'winning', 'yet', 'feel', 'vindicated', 'centipedes', 'nothing', 'future', 'history', 'part', 'history', 'thedonald', 'part', 'history', 'knew', 'hearts', 'one', 'believed', 'us', 'always', 'knew', 'truth', 'brakes', 'train', 'trump', 'wins'] 

['hello', 'thedonald', 'readers', 'entire', 'reddit', 'community', 'going', 'huge', 'im', 'looking', 'forward', 'answering', 'questions', 'im', 'flight', 'visit', 'great', 'people', 'toledo', 'oh', 'internet', 'connection', 'might', 'spotty', 'promise', 'ill', 'answer', 'questions', 'want', 'big', 'things', 'america', 'president', 'make', 'americ